##### Copyright 2020 The TensorFlow IO Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Almacenamiento de blobs de Azure con TensorFlow

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/io/tutorials/azure"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/io/tutorials/azure.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/io/tutorials/azure.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver código fuente en GitHub</a>
</td>
      <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/io/tutorials/azure.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar el bloc de notas</a>
</td>
</table>

Advertencia: Además de los paquetes de Python, este cuaderno usa `npm install --user` para instalar los paquetes. Tenga cuidado al ejecutar localmente.


## Descripción general

Este tutorial muestra cómo usar archivos de lectura y escritura en [Azure Blob Storage](https://azure.microsoft.com/en-us/services/storage/blobs/) con TensorFlow, a través de la integración del sistema de archivos Azure de TensorFlow IO.

Se necesita una cuenta de Azure Storage para leer y escribir archivos en Azure Blob Storage. La clave de Azure Storage se debe proporcionar a través de la variable del entorno:

```
os.environ['TF_AZURE_STORAGE_KEY'] = '&lt;key&gt;'
```

El nombre de la cuenta de almacenamiento y el nombre del contenedor son parte del nombre de archivo uri:

```
azfs://&lt;storage-account-name&gt;/&lt;container-name&gt;/&lt;path&gt;
```

En este tutorial, para fines de demostración, puede configurar opcionalmente [Azurite](https://github.com/Azure/Azurite), que es un emulador de Azure Storage. Con el emulador de Azurite es posible leer y escribir archivos a través de la interfaz de Azure Blob Storage con TensorFlow.

## Preparación y uso

### Instale los paquetes necesarios y reinicie el tiempo de ejecución

In [2]:
try:
  %tensorflow_version 2.x 
except Exception:
  pass

!pip install tensorflow-io

TensorFlow 2.x selected.
     |████████████████████████████████| 20.1MB 42.7MB/s 


### Instalar y preparar Azurite (opcional)

En caso de que una cuenta de Azure Storage no esté disponible, se necesita lo siguiente para instalar y configurar Azurite que emula la interfaz de Azure Storage:

In [3]:
!npm install azurite@2.7.0

npm WARN deprecated request@2.87.0: request has been deprecated, see https://github.com/request/request/issues/3142
npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ azurite@2.7.0
added 116 packages from 141 contributors in 6.591s


In [4]:
# The path for npm might not be exposed in PATH env,
# you can find it out through 'npm bin' command
npm_bin_path = get_ipython().getoutput('npm bin')[0]
print('npm bin path: ', npm_bin_path)

# Run `azurite-blob -s` as a background process. 
# IPython doesn't recognize `&` in inline bash cells.
get_ipython().system_raw(npm_bin_path + '/' + 'azurite-blob -s &')

npm bin path:  /content/node_modules/.bin


### Leer y escribir archivos en Azure Storage con TensorFlow

El siguiente es un ejemplo de lectura y escritura de archivos en Azure Storage con la API de TensorFlow.

Se comporta de la misma manera que otros sistemas de archivos (por ejemplo, POSIX o GCS) en TensorFlow una vez que se importa el paquete `tensorflow-io`, ya que `tensorflow-io` registrará automáticamente el esquema `azfs` para su uso.

La clave de Azure Storage se debe proporcionar a través de la variable del entorno `TF_AZURE_STORAGE_KEY`. De lo contrario, `TF_AZURE_USE_DEV_STORAGE` podría establecerse en `True` para usar el emulador Azurite en su lugar:


In [ ]:
import os
import tensorflow as tf
import tensorflow_io as tfio

# Switch to False to use Azure Storage instead:
use_emulator = True

if use_emulator:
  os.environ['TF_AZURE_USE_DEV_STORAGE'] = '1'
  account_name = 'devstoreaccount1'
else:
  # Replace <key> with Azure Storage Key, and <account> with Azure Storage Account
  os.environ['TF_AZURE_STORAGE_KEY'] = '<key>'
  account_name = '<account>'

  # Alternatively, you can use a shared access signature (SAS) to authenticate with the Azure Storage Account
  os.environ['TF_AZURE_STORAGE_SAS'] = '<your sas>'
  account_name = '<account>'

In [6]:
pathname = 'az://{}/aztest'.format(account_name)
tf.io.gfile.mkdir(pathname)

filename = pathname + '/hello.txt'
with tf.io.gfile.GFile(filename, mode='w') as w:
  w.write("Hello, world!")

with tf.io.gfile.GFile(filename, mode='r') as r:
  print(r.read())

Hello, world!


## Configuraciones

Las configuraciones de Azure Blob Storage en TensorFlow siempre se realizan a través de variables del entorno. A continuación se muestra una lista completa de las configuraciones disponibles:

- `TF_AZURE_USE_DEV_STORAGE`: se debe establecer en 1 para usar el emulador de almacenamiento de desarrollo local para conexiones como 'az://devstoreaccount1/container/file.txt'. Esto tendrá prioridad sobre todas las demás configuraciones, por eso se puede anular (`unset`) para usar cualquier otra conexión.
- `TF_AZURE_STORAGE_KEY`: clave de cuenta para la cuenta de almacenamiento en uso
- `TF_AZURE_STORAGE_USE_HTTP`: se debe establecer en cualquier valor si no se desea usar la transferencia https. Se puede anular (`unset`) para usar el valor predeterminado https
- `TF_AZURE_STORAGE_BLOB_ENDPOINT`: se debe establecer en el punto final del almacenamiento de blobs; el valor predeterminado es `.core.windows.net`.
